In [3]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import SGDClassifier, Perceptron #doubt linear models work, linear reg.
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.model_selection import train_test_split
import numpy.random as nr
nr.seed = 42

In [5]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path as op
import itertools
import re
import xarray as xr

%matplotlib inline

In [6]:
inputPath = '/scratch/adomakor412/all_npy3'
PATH = os.path.expanduser(inputPath)
ncPath = os.path.expanduser('/scratch/adomakor412/april_data_2019')

In [7]:
columns = [
    "time",
    "longitude",
    "latitude",
    #"band",#Necessary?
    "G17_Temp",
    "G17_mean",
    "G17_std",#mean and std outside inner for loop for comp. eff.
    "target_G16_Temp"
]

df = pd.DataFrame({},columns = columns)


,time,longitude,latitude,G17_Temp,G17_mean,G17_std,target_G16_Temp


In [8]:
def longitude(lonMin, lonMax, col, colSize):
    lon = (col/colSize)* (lonMax - lonMin)
    return lon

def latitude(latMin, latMax, row, rowSize):
    lat = (row/rowSize)* (latMax - latMin)
    return lat

def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [9]:
import logging
logger = logging.getLogger()

In [ ]:
extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]

with open('logML', 'a') as log:   
    for (bb,dd) in list(itertools.product([8],[5])):#Let's start with one day
        DD = str(98+dd).zfill(3)
        lookup = f'M6C08_G17_s2019{DD}'
        ncFiles = [f for f in os.listdir(ncPath) if re.search(lookup,f)].sort()
        npFiles = [f for f in os.listdir( PATH ) if re.search(lookup,f)].sort()



        for ncf, npf in zip(ncFiles,npFiles):
            try:
                imageBox = np.load(op.join( PATH,npf))
                myFile = xr.open_dataset(op.join(ncPath,ncf))
                planck_fk1 = float(myFile['planck_fk1'].data)
                planck_fk2 = float(myFile['planck_fk2'].data) 
                planck_bc1 = float(myFile['planck_bc1'].data)                       
                planck_bc2 = float(myFile['planck_bc2'].data)

                time = ncf[31:38]
                G17_mean = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
                G17_std = Rad2BT(imageBox.std(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)

                hh = ncf[34:36]
                mm = ncf[36:38]
                
                print(ncf)
                print(str(ncf), file=log)
                logger.info(str(ncf))

                G16_npy = np.load( op.join(PATH, npf.replace('G16','G17',1)) )
                G16_ncf = xr.open_dataset(op.join( ncPath, ncf.replace('G16','G17',1) ))
                G16_fk1 = float(G16_ncf['planck_fk1'].data)
                G16_fk2 = float(G16_ncf['planck_fk2'].data) 
                G16_bc1 = float(G16_ncf['planck_bc1'].data)                       
                G16_bc2 = float(G16_ncf['planck_bc2'].data)

                target_G16_Temp = Rad2BT(G16_npy, G16_fk1, G16_fk2, G16_bc1, G16_bc2)
                x,y = imageBox.shape[0],imageBox.shape[1]
                for i,j in itertools.product(range(x),range(y)):
                    lon = longitude( extent_pc[0], extent_pc[1], i, x )
                    lat = latitude( extent_pc[2], extent_pc[3], j, y)

                    G17_Temp = Rad2BT(imageBox[i,j], planck_fk1, planck_fk2, planck_bc1, planck_bc2)#unfiltered
                    target_G16_Temp = Rad2BT( G16_npy[i,j], G16_fk1, G16_fk2, G16_bc1, G16_bc2 )
                    
                    row = [time, lon, lat, G17_Temp, G17_mean, G17_std, target_G16_Temp]

                    df = df.append(pd.DataFrame([row],columns=columns),ignore_index=True)
                    
            except ValueError as e:
                logger.exception(e)
                print(e)
                print(e, file=log)
                continue 
    
# df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=columns)
# for column in df.columns:
#     if df[column].dtype == "object":
#         df[column] = df[column].str.strip()
# df = df.dropna()

OR_ABI-L1b-RadF-M6C08_G17_s20191030640339_e20191030649405_c20191030649442.nc


In [ ]:
dfSample = df.dropna().sample(n=1000)
X = dfSample.drop(["target_G16_Temp"],axis=1)
y = dfSample["target_G16_Temp"]

https://erikbern.com/2018/02/15/new-benchmarks-for-approximate-nearest-neighbors.html

In [ ]:
# Get baseline results here with logisic regression and random forest
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression
#REGRESSION TREE B/C IF TIME IS IN WINDOW OF ANOMALY IT WILL DO ONE THING OTHERWISE ANOTHER DO ANOTHER THING
#Try KNN apx

# Set up your models here

def model_one():
    pass

def model_two():
    pass

def model_three():
    pass

def model_four():
    pass

def model_five():
    pass

heldout = [0.95, 0.90, 0.75, 0.50, 0.01]
rounds = 20

classifiers = [
    ("SGD", SGDClassifier(max_iter=100)),
    ("ASGD", SGDClassifier(average=True)),
    ("Perceptron", Perceptron()),
    ("Passive-Aggressive I", PassiveAggressiveClassifier(loss='hinge',
                                                         C=1.0, tol=1e-4)),
    ("Passive-Aggressive II", PassiveAggressiveClassifier(loss='squared_hinge',
                                                          C=1.0, tol=1e-4)),
    ("SAG", LogisticRegression(solver='sag', tol=1e-1, C=1.e4 / X.shape[0]))
]

xx = 1. - np.array(heldout)

for name, clf in classifiers:
    print("training %s" % name)
    rng = np.random.RandomState(42)
    yy = []
    for i in heldout:
        yy_ = []
        for r in range(rounds):
            X_train, X_test, y_train, y_test = \
                train_test_split(X, y, test_size=i, random_state=rng)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            yy_.append(1 - np.mean(y_pred == np.array(y_test)))
        yy.append(np.mean(yy_))
    plt.plot(xx, yy, label=name)

plt.legend(loc="upper right")
plt.xlabel("Proportion train")
plt.ylabel("Test Error Rate")
plt.show()

In [ ]:
# Part 4 inplement cross validation here
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)

xx = ['K_fold-1','K_fold-2','K_fold-3','K_fold-4','K_fold-5']
rounds = 20
#count = 0
for train_index, test_index in kf.split(X):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = np.array(X)[np.array(train_index)], np.array(X)[np.array(test_index)]
     y_train, y_test = np.array(y)[np.array(train_index)], np.array(y)[np.array(test_index)]
        
     classifiers = [
     ("ASGD", SGDClassifier(average=True)),
     ("SAG", LogisticRegression(solver='sag', tol=1e-1, C=1.e4 / X.shape[0]))
     ]
     #count+=1

     for name, clf in classifiers:
         print("training %s" % name)
         rng = np.random.RandomState(42)
         yy = []
         for i in heldout:
             yy_ = []
             for r in range(rounds):
                 clf.fit(X_train, y_train)
                 y_pred = clf.predict(X_test)
                 yy_.append(1 - np.mean(y_pred == np.array(y_test)))
             yy.append(np.mean(yy_))
         plt.plot(xx, yy, label=name)

     plt.legend(loc="upper right")
     plt.xlabel("Train-Test Index")
     plt.ylabel("Test Error Rate")
     plt.show()